# Welcome to pandas demo

### by Jozef Karabelly


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import string
from datetime import time

%matplotlib inline
AAPL_PATH = './data/NASDAQ_AAPL.txt'
PATH = './data/%s.csv'

def load_bars(ticker):
    bars = pd.read_csv(PATH % ticker)
    bars.index = pd.to_datetime(bars.pop('Date'))
    return bars

def subsample(frame, pct=.9):
    N = len(frame)
    indexer = np.sort(np.random.permutation(N)[:int(pct*N)])
    return frame.take(indexer)

def rands(n):
    choices = string.ascii_letters
    return ''.join([random.choice(choices) for _ in range(n)])

tsla_bars = load_bars('TSLA')
ind_names = np.array(['ENERGY', 'FINANCIAL', 'TECH', 'CONSDUR', 'SERVICES', 'UTILITIES'], dtype='O')
ccys = np.array(['USD', 'EUR'], dtype='O')
Nfull = 2000
tickers = np.array(sorted(rands(5).upper() for _ in range(Nfull)), dtype='O')
tickers = np.unique(tickers)

industries = pd.Series(ind_names.take(np.random.randint(0, 6, Nfull)), index=tickers, name='industry')
ccy = pd.Series(ccys.take(np.random.randint(0, len(ccys), Nfull)), index=tickers, name='ccy')
zscore = lambda x: (x - x.mean()) / x.std()

base = './data/movielens'
get_path = lambda x: f'{base}/{x}.dat'

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv(get_path('users'), sep='::', names=unames, header=None)

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(get_path('ratings'), sep='::', names=rnames, header=None)

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv(get_path('movies'), sep='::', names=mnames, header=None)

/home/jkarabelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/home/jkarabelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/home/jkarabelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


<div align="center">
  <img src="https://dev.pandas.io/static/img/pandas.svg" width="300px"><br>
</div>


# What is pandas?

# Main features
* Time series functionality
* Data alignment
* Missing data
* Groupby operation
* Merging and joining
* Data summary


## Load tabular data

In [3]:
aapl = pd.read_csv(AAPL_PATH)
aapl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 7 columns):
Ticker    2014 non-null object
Date      2014 non-null int64
Open      2014 non-null float64
High      2014 non-null float64
Low       2014 non-null float64
Close     2014 non-null float64
Vol       2014 non-null int64
dtypes: float64(4), int64(2), object(1)
memory usage: 110.2+ KB


## Time series operations

In [ ]:
aapl.index = pd.to_datetime(aapl.pop('Date'), format='%Y%m%d%H%M')
aapl.head()

In [ ]:
aapl.at_time(time(15, 0)).head(10)

In [ ]:
aapl.Close['2010-11-01'].head()

In [ ]:
month_agg = aapl.Close.resample('M').agg(['median', 'mean', 'std'])
month_agg

In [ ]:
returns = aapl.Close.pct_change()
std_day = returns.rolling(95).std()
std_day.resample('B').plot();

## Data alignment

In [ ]:
ts1 = pd.Series(np.random.rand(10), index=pd.date_range('2010-10-10', periods=10))
ts1

In [ ]:
ts2 = ts1[[0, 2, 4, 5, 7, 8, 9]]
ts2

In [ ]:
ts1 + ts2

In [ ]:
df = pd.DataFrame({'A': ts1, 'B': ts2})
df

In [ ]:
amd_bars = load_bars('AMD')

In [ ]:
f1 = subsample(amd_bars)
f2 = subsample(tsla_bars)
f1.info()

In [ ]:
both = pd.concat([f1, f2], axis=1, keys=['AMD', 'TSLA'])
both.head()

## Missing data

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df.sum()

In [ ]:
df.mean(1)

In [ ]:
df.dropna()

In [ ]:
df.fillna(0) # method='ffill'

## Groupby operation

In [ ]:
df = pd.DataFrame({'Momentum': np.random.randn(1000) / 200 + 0.03,
                   'Value': np.random.randn(1000) / 200 + 0.08,
                   'ShortInterest': np.random.randn(1000) / 200 - 0.02},
                 index=tickers.take(np.random.permutation(Nfull)[:1000]))
df.head()

In [ ]:
means = df.groupby(industries).mean()
means.plot(kind='barh');

In [ ]:
means = df.groupby([industries, ccy]).mean()
means

In [ ]:
normed = df.groupby([industries, ccy]).apply(zscore)
normed.groupby([industries, ccy]).agg(['mean', 'std'])

## Hierarchical indexing

In [ ]:
means

In [ ]:
means['Momentum']

In [ ]:
means.loc['TECH']

In [ ]:
means.stack()

In [ ]:
means.stack().unstack('industry')

## Merging and joining

In [ ]:
movies.head()  

In [ ]:
ratings.head()

In [ ]:
users.head()

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)
data.info()

In [ ]:
rating_counts = data.groupby('title').size()
freq_titles = rating_counts.index[rating_counts > 1000]

In [ ]:
highest_rated = data.groupby('title').rating.mean()[freq_titles].sort_values()[-10:]
highest_rated

## Pivot tables

In [ ]:
mean_ratings = data.pivot_table('rating', index='title', columns='gender', aggfunc='mean')
mean_ratings.tail(10)

## Data summary and statistics

In [ ]:
data.title.value_counts().head(10)

In [ ]:
data.rating.describe()

In [ ]:
by_gender = data.groupby('gender').rating.describe()
by_gender

In [ ]:
by_gender.stack().unstack(0)

# Questions